In [71]:
import pandas as pd
from datetime import date

In [72]:
clean_sample_sheet = '/Users/jolsen98/Desktop/GSS_atacseq/SampleSheet_clean_01_19_24.csv'
tracking_sheet = '/Users/jolsen98/Desktop/GSS_atacseq/GSS SAMPLE TRACKER_01_23_24.xlsx'
dual_index_primer_list = '/Users/jolsen98/Desktop/GSS_atacseq/ATAC dual-indexed primer list_96x.xlsx'

In [73]:
outpath = '/Users/jolsen98/Desktop/GSS_atacseq/{}_demux_sample_sheet.csv'.format(date.today().strftime("%Y-%m-%d"))

In [74]:
samplesheet_header = []
samplesheet_columns = []
with open(clean_sample_sheet, 'r') as f:
    for line in f:
        if all(item in line for item in ["Sample_ID", "Sample_Name", "Sample_Well"]):
            samplesheet_columns = line.strip('\n').split(",")
            break
        else:
            samplesheet_header.append(line)

In [75]:
df_ts = pd.read_excel(tracking_sheet, sheet_name='ATACSEQ')
df_ts = df_ts[['GSS ID','QUIBIT WELL NAME']]

In [76]:
df_dipl = pd.read_excel(dual_index_primer_list, sheet_name='96x-1 byRow')
df_dipl = df_dipl[["Plate","Position", 'Index 1', 'Index 2', 'Index 1.1', 'Index 2_iseq_novaseq', 'Index 2.1' ]]

In [77]:
merged = df_ts.merge(df_dipl, left_on = 'QUIBIT WELL NAME', right_on = 'Position')

In [78]:
merged

,GSS ID,QUIBIT WELL NAME,Plate,Position,Index 1,Index 2,Index 1.1,Index 2_iseq_novaseq,Index 2.1
0,GSS253639,A1,96x-1,A1,i7-1,i5-1,TAAGGCGA,TAGATCGC,GCGATCTA
1,GSS153634,B1,96x-1,B1,i7-1,i5-2,TAAGGCGA,CTCTCTAT,ATAGAGAG
2,GSS153637,C1,96x-1,C1,i7-1,i5-3,TAAGGCGA,TATCCTCT,AGAGGATA
3,GSS215019,D1,96x-1,D1,i7-1,i5-4,TAAGGCGA,AGAGTAGA,TCTACTCT
4,GSS115014,E1,96x-1,E1,i7-1,i5-5,TAAGGCGA,GTAAGGAG,CTCCTTAC
...,...,...,...,...,...,...,...,...,...
59,GSS288689,D8,96x-1,D8,i7-8,i5-4,CAGAGAGG,AGAGTAGA,TCTACTCT
60,GSS276239,E8,96x-1,E8,i7-8,i5-5,CAGAGAGG,GTAAGGAG,CTCCTTAC
61,GSS224169,F8,96x-1,F8,i7-8,i5-6,CAGAGAGG,ACTGCATA,TATGCAGT
62,GSS270149,G8,96x-1,G8,i7-8,i5-7,CAGAGAGG,AAGGAGTA,TACTCCTT


In [79]:
# Index 2_iseq_novaseq is demultiplex column

In [80]:
idx2_demux_col = 'Index 2_iseq_novaseq'

In [81]:
merged = merged[['GSS ID','Plate','Position', 'Index 1', 'Index 2', 'Index 1.1', idx2_demux_col]]

In [82]:
merged

,GSS ID,Plate,Position,Index 1,Index 2,Index 1.1,Index 2_iseq_novaseq
0,GSS253639,96x-1,A1,i7-1,i5-1,TAAGGCGA,TAGATCGC
1,GSS153634,96x-1,B1,i7-1,i5-2,TAAGGCGA,CTCTCTAT
2,GSS153637,96x-1,C1,i7-1,i5-3,TAAGGCGA,TATCCTCT
3,GSS215019,96x-1,D1,i7-1,i5-4,TAAGGCGA,AGAGTAGA
4,GSS115014,96x-1,E1,i7-1,i5-5,TAAGGCGA,GTAAGGAG
...,...,...,...,...,...,...,...
59,GSS288689,96x-1,D8,i7-8,i5-4,CAGAGAGG,AGAGTAGA
60,GSS276239,96x-1,E8,i7-8,i5-5,CAGAGAGG,GTAAGGAG
61,GSS224169,96x-1,F8,i7-8,i5-6,CAGAGAGG,ACTGCATA
62,GSS270149,96x-1,G8,i7-8,i5-7,CAGAGAGG,AAGGAGTA


In [83]:
merged.rename(columns = {'GSS ID': "Sample_Name","Plate":"Sample_Plate","Position":"Sample_Well",
                         "Index 1": "I7_Index_ID", "Index 2": "I5_Index_ID", "Index 1.1": "index", 
                         idx2_demux_col:"index2"}, inplace=True)

/var/folders/l4/dlk78yxs5tngl7cb382dnvpc0000gr/T/ipykernel_27814/1178162519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns = {'GSS ID': "Sample_Name","Plate":"Sample_Plate","Position":"Sample_Well",


In [84]:
merged['Sample_ID'] = merged["Sample_Name"]

/var/folders/l4/dlk78yxs5tngl7cb382dnvpc0000gr/T/ipykernel_27814/1909009255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['Sample_ID'] = merged["Sample_Name"]


In [85]:
merged["Sample_Project"] = "GSS_ATACSEQ_RUN1"

In [86]:
merged["Description"] = ""

In [89]:
merged = merged[samplesheet_columns]

In [90]:
merged

,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description
0,GSS253639,GSS253639,96x-1,A1,i7-1,TAAGGCGA,i5-1,TAGATCGC,GSS_ATACSEQ_RUN1,
1,GSS153634,GSS153634,96x-1,B1,i7-1,TAAGGCGA,i5-2,CTCTCTAT,GSS_ATACSEQ_RUN1,
2,GSS153637,GSS153637,96x-1,C1,i7-1,TAAGGCGA,i5-3,TATCCTCT,GSS_ATACSEQ_RUN1,
3,GSS215019,GSS215019,96x-1,D1,i7-1,TAAGGCGA,i5-4,AGAGTAGA,GSS_ATACSEQ_RUN1,
4,GSS115014,GSS115014,96x-1,E1,i7-1,TAAGGCGA,i5-5,GTAAGGAG,GSS_ATACSEQ_RUN1,
...,...,...,...,...,...,...,...,...,...,...
59,GSS288689,GSS288689,96x-1,D8,i7-8,CAGAGAGG,i5-4,AGAGTAGA,GSS_ATACSEQ_RUN1,
60,GSS276239,GSS276239,96x-1,E8,i7-8,CAGAGAGG,i5-5,GTAAGGAG,GSS_ATACSEQ_RUN1,
61,GSS224169,GSS224169,96x-1,F8,i7-8,CAGAGAGG,i5-6,ACTGCATA,GSS_ATACSEQ_RUN1,
62,GSS270149,GSS270149,96x-1,G8,i7-8,CAGAGAGG,i5-7,AAGGAGTA,GSS_ATACSEQ_RUN1,


In [92]:
columns_str = ",".join(samplesheet_columns)+'\n'

with open(outpath,'w') as f:
    for line in samplesheet_header:
        f.write(line)
    f.write(columns_str)
f.close()
merged.to_csv(outpath,mode='a', header=False, index=False)

